# DSC 80: Lab 04

### Due Date: Saturday October 31th, Midnight (11:59 PM)

## Zoom Lab Hours
- Follow instructions on this link: https://docs.google.com/document/d/16qZpPSYhxwQDMcn-lGQjC-J-PzppLevv_mANLt2ko8g/edit 

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding work will be developed in an accompanying `lab*.py` file, that will be imported into the current notebook.

Labs and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying python file will be tested (a la DSC 20),
2. The notebook will be graded (for graphs and free response questions).

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for working in the Notebook**:
- The notebooks serve to present you the questions and give you a place to present your results for later review.
- The notebook on *lab assignments* are not graded (only the `.py` file).
- Notebooks for PAs will serve as a final report for the assignment, and contain conclusions and answers to open ended questions that are graded.
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `.py` file.

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional functions to solve the lab! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `lab**.py` (much like we do in the notebook).
- Always document your code!

### Importing code from `lab**.py`

* We import our `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab**.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab**.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab**.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab**` merely import the existing compiled python.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lab04 as lab

In [3]:
import os
import pandas as pd
import numpy as np

## Login time questions

Imagine that you own an online store and you'd like to monitor the visits to your site. You've collected some data that you store in login_table.csv. It contains the information about different login dates and times for different users. Some users are unique, some visited your store multiple times.

You need to answer a few questions below in order to understand the login patters of your users.

In [4]:
fp = os.path.join('data', 'login_table.csv')
login = pd.read_csv(fp)

**Question 1**

Write a function `latest_login` which takes in a dataframe like `login` and outputs a dataframe, indexed by `Login Id`, of the login that occurs at the latest time-of-day for each user. Latest time-of-day is as it says: the latest time, **regardless of how recent the day is**.

For example, if a user always logs in once per day at Noon, but her most recent log in happened to be at 8:00PM, then her latest log-in time becomes 8:00PM.

Note: you do not need to import datetime. Look into how pandas has its own built in methods for dates.

In [23]:
def latest_login(login):
    """Calculates the latest login time for each user
    :param login: a dataframe with login information
    :return: a dataframe with latest login time for
    each user indexed by "Login Id"
    >>> fp = os.path.join('data', 'login_table.csv')
    >>> login = pd.read_csv(fp)
    >>> result = latest_login(login)
    >>> len(result)
    433
    >>> result.loc[381, "Time"].hour > 12
    True
    """
    df = login.copy()
    df['Time'] = login['Time'].apply(pd.to_datetime).dt.time
    return df.groupby('Login Id').max()

In [24]:
fp = os.path.join('data', 'login_table.csv')
login = pd.read_csv(fp)
result = latest_login(login)
result

,Time
Login Id,
381,22:51:15
393,23:47:12
412,19:49:10
413,23:39:23
419,16:37:33
...,...
1302,19:38:41
1304,15:51:40
1305,17:45:11


**Question 2**

As a site owner, you would like to see how often users return to your site. You've noticed that there are users who have several logins and users who logged in only once. Of those users that logged in more than once, you are interested in finding the shortest amount of time elapsed between two consecutive logins for each of these users.

Write a function `smallest_ellapsed` which takes in a dataframe like `login` and outputs a dataframe, indexed by Login ID, containing the shortest time elapsed for each user. Any users who haven't logged in more than once should not be included in the output.

In [71]:
def smallest_ellapsed(login):
    """
    Calculates the the smallest time elapsed for each user.
    :param login: a dataframe with login information but without unique IDs
    :return: a dataframe, indexed by Login ID, containing 
    the smallest time elapsed for each user.
    >>> fp = os.path.join('data', 'login_table.csv')
    >>> login = pd.read_csv(fp)
    >>> result = smallest_ellapsed(login)
    >>> len(result)
    238
    >>> 18 < result.loc[1233, "Time"].days < 23
    True
    """
    df = login.copy()
    df['Time'] = df['Time'].apply(pd.to_datetime)
    out = df.groupby('Login Id').apply(lambda x : x.sort_values('Time').diff().min())
    return out.drop(columns = 'Login Id').dropna()

In [72]:
fp = os.path.join('data', 'login_table.csv')
login = pd.read_csv(fp)
result = smallest_ellapsed(login)
result

,Time
Login Id,
381,00:00:00
393,00:00:19
412,00:00:14
413,00:00:00
419,00:25:16
...,...
1291,23:05:49
1302,06:52:55
1304,22:14:43


# Pivot tables

**Question 3**

The pivot table allows you to group the entries of a dataframe into a two-dimensional table that provides a (multidimensional) summarization of the data. You are given a simple dataset, `sales.csv`, and are asked to solve a few simple problems using a `pivot table`.  

We have provided the outline for your pivot tables. Your values will be different.

* Write a function `total_seller` that takes `sales` dataframe and returns a pivot table that contains a total for each seller, indexed by a name.

<img src="imgs/image_0.png" width="15%"/>

* Write a function `product_name` that takes `sales` dataframe and returns a pivot table that contains a total for each seller, indexed by a product.

<img src="imgs/image_1.png" width="25%"/>

* Write a function `count_product` that takes `sales` dataframe and returns a pivot table that contains the total amount of items sold product wise, name wise per date. Replaces `NaNs` with 0s. 

<img src="imgs/image_2.png" width="35%"/>

* Write a function `total_by_month` that takes `sales` dataframe and returns a pivot table that contains the total amount name wise, product wise per `month`. Replaces `NaNs` with 0s.

<img src="imgs/image_3.png" width="40%"/>


Note: Here is a <a href = "https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html"> link </a> to a great source that provides an overview of the pivot tables with mane examples from the Titanic dataset. 

If your "Total" title is on the right side of the DataFrame instead of the left as seen above, as long as you are passing the doctests you should be fine. This is likely due to an issue with pandas versions.

In [48]:
sales_fp = os.path.join('data', 'sales.csv')
sales = pd.read_csv(sales_fp)
sales

,Name,Product,Date,Total
0,Smith,book,01.01.2012,200
1,Jones,pen,02.20.2013,300
2,Trump,hotel,03.03.2015,100
3,Smith,book,05.10.2013,2000
4,Jones,book,02.20.2013,30
5,Trump,boat,03.30.2017,700
6,Smith,ruler,07.05.2014,2100
7,Jones,ruler,02.25.2015,350
8,Trump,book,03.03.2015,1000
9,Smith,pen,01.01.2012,2500


In [94]:
total_by_month(sales)

Month          February  January  July  March  May
Name  Product                                     
Jones book            1        0     0      0    0
      pen             1        0     0      0    0
      ruler           2        0     0      0    0
Smith book            0        1     0      0    1
      pen             0        1     0      0    0
      ruler           0        0     1      0    0
Trump boat            0        0     0      1    0
      book            0        0     0      1    0
      hotel           0        0     0      1    0
      pen             0        0     0      1    0

In [243]:
def total_seller(df):
    """
    Total for each seller
    :param df: like sales
    :return: pivot table
    >>> fp = os.path.join('data', 'sales.csv')
    >>> df = pd.read_csv(fp)
    >>> out = total_seller(df)
    >>> out.index.dtype
    dtype('O')
    >>> out["Total"].sum() < 15000
    True

    """
    
    return pd.pivot_table(df, index = 'Name', aggfunc = 'sum')


def product_name(df):
    """
    :param df: like sales
    :return: pivot table
    >>> fp = os.path.join('data', 'sales.csv')
    >>> df = pd.read_csv(fp)
    >>> out = product_name(df)
    >>> out.size
    15
    >>> out.loc["pen"].isnull().sum()
    0
    """
    
    return pd.pivot_table(df, index = 'Product', columns = 'Name', aggfunc = 'sum')

def count_product(df):
    """
    :param df: like sales
    :return: pivot table
    >>> fp = os.path.join('data', 'sales.csv')
    >>> df = pd.read_csv(fp)
    >>> out = count_product(df)
    >>> out.loc["boat"].loc["Trump"].value_counts()[0]
    6
    >>> out.size
    70
    """
    
    return pd.pivot_table(df, index = ['Product','Name'], columns = 'Date',aggfunc = 'count', fill_value = 0)


def total_by_month(df):
    """
    :param df: like sales
    :return: pivot table
    >>> fp = os.path.join('data', 'sales.csv')
    >>> df = pd.read_csv(fp)
    >>> out = total_by_month(df)
    >>> out["Total"]["May"].idxmax()
    ('Smith', 'book')
    >>> out.shape[1]
    5
    """
    df['Month'] = df['Date'].apply(pd.to_datetime).dt.month
    rep_dic = {1:'January', 2:'February',3:'March',4:'April', 5:'May', 6:'June', 7:'July',8:'August', 9:'September', 10:'October',11:'November',12:'December'}
    df['Month'] = df['Month'].replace(rep_dic)
    return pd.pivot_table(df, index = ['Name', 'Product'], columns = 'Month', aggfunc = 'count', fill_value = 0).drop(columns = 'Date')

In [246]:
fp = os.path.join('data', 'sales.csv')
df = pd.read_csv(fp)
out = total_seller(df)
out

,Total
Name,
Jones,3680
Smith,6800
Trump,1950


# A distribution of Skittles

[Skittles](https://en.wikipedia.org/wiki/Skittles_(confectionery)) are made in two locations in the United States: Yorkville, Illinois and Waco, Texas. In these factories, Skittles of different colors are made separately by different machines and combined/packaged into bags for sale. The tab-separated file `skittles.tsv` contains the contents of 468 bags of Skittles.

Most people have preferences for their favorite flavor and there is a surprising amount of variation among the distribution of flavors in each bag.

Look at the variation by bag in the dataset below:

In [95]:
skittles_fp = os.path.join('data', 'skittles.tsv')
skittles = pd.read_csv(skittles_fp, sep='\t')

In [96]:
skittles

,red,orange,yellow,green,purple,Factory
0,10,15,11,7,18,Yorkville
1,5,12,17,15,10,Yorkville
2,16,11,15,11,9,Waco
3,15,8,13,16,7,Waco
4,11,14,20,8,7,Waco
...,...,...,...,...,...,...
463,11,11,12,13,11,Waco
464,17,10,8,11,12,Waco
465,9,14,12,10,15,Waco
466,12,14,11,10,10,Waco


### Differences between Yorkville and Waco

**Question 4**

First, you will investigate if the machine that mixes together the Skittles of different colors might favor one color over another. Use a permutation test to assess whether, on average, bags made in Yorkville have the same number of orange skittles as bags made in Waco. Do this by following the outline below:
1. Create a function `diff_of_means` that takes in a dataframe of counts of skittles (like `skittles`) and their origin and returns the *absolute* difference of means between the number of orange Skittles per bag from Yorkville and Waco.
2. Create a function `simulate_null` that takes in a dataframe of counts of skittles (like `skittles`) and their origin, and returns one instance of the test-statistic under the null hypothesis.
3. Create a function `pval_orange` that takes in a dataframe of counts of skittles (like `skittles`) and their origin, and calculates the p-value for the permutation test using `1000` trials.

Plot the observed statistic, along with the histogram for the simulated distribution, to check your work.

In [141]:
def diff_of_means(data, col='orange'):
    """
    diff_of_means takes in a dataframe of counts 
    of skittles (like skittles) and their origin 
    and returns the absolute difference of means 
    between the number of oranges per bag from Yorkville and Waco.

    :Example:
    >>> skittles_fp = os.path.join('data', 'skittles.tsv')
    >>> skittles = pd.read_csv(skittles_fp, sep='\\t')
    >>> out = diff_of_means(skittles)
    >>> 0 <= out
    True
    """
    
    return abs(np.diff(data.groupby('Factory').mean()[col].values)[0])


def simulate_null(data, col='orange'):
    """
    simulate_null takes in a dataframe of counts of 
    skittles (like skittles) and their origin, and 
    generates one instance of the test-statistic 
    under the null hypothesis

    :Example:
    >>> skittles_fp = os.path.join('data', 'skittles.tsv')
    >>> skittles = pd.read_csv(skittles_fp, sep='\\t')
    >>> out = simulate_null(skittles)
    >>> isinstance(out, float)
    True
    >>> 0 <= out <= 1.0
    True
    """
    df = data.copy()
    df[col] = df[col].sample(replace = True, frac = 1).reset_index(drop=True)
    return abs(np.diff(df.groupby('Factory').mean()[col].values)[0])


def pval_orange(data, col='orange'):
    """
    pval_orange takes in a dataframe of counts of 
    skittles (like skittles) and their origin, and 
    calculates the p-value for the permutation test 
    using 1000 trials.
    
    :Example:
    >>> skittles_fp = os.path.join('data', 'skittles.tsv')
    >>> skittles = pd.read_csv(skittles_fp, sep='\\t')
    >>> pval = pval_orange(skittles)
    >>> isinstance(pval, float)
    True
    >>> 0 <= pval <= 0.1
    True
    """
    n = 1000
    t_stats = []
    for i in range(n):
        t_stats.append(simulate_null(data))
    obs = diff_of_means(data, col = col)
    return np.count_nonzero(np.array(t_stats) >= obs) / n

In [142]:
skittles_fp = os.path.join('data', 'skittles.tsv')
skittles = pd.read_csv(skittles_fp, sep='\\t')
out = pval_orange(skittles)
out

<ipython-input-142-065aedb690dd>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  skittles = pd.read_csv(skittles_fp, sep='\\t')


0.059

In [146]:
skittles.groupby('Factory').mean().diff()

,red,orange,yellow,green,purple
Factory,,,,,
Waco,NaN,NaN,NaN,NaN,NaN
Yorkville,-0.356085,-0.60044,1.216716,-0.215103,-0.009311


**Question 5**

Use your work from above to decide which colors tend to differ the most between the two locations on average. Create a function `ordered_colors` that returns your answer as an ordered list from "most different" to "least different" between the two locations. Your list should be a *hard-coded* list, where each element has the form `(color, p-value)`.

Even though there is randomness in the color composition in each bag, this list gives the likelihood that the machines have a systematic, meaningful, difference in how they blend the colors in each bag.

In [147]:
def ordered_colors():
    """
    ordered_colors returns your answer as an ordered
    list from "most different" to "least different" 
    between the two locations. You list should be a 
    hard-coded list, where each element has the 
    form (color, p-value).

    :Example:
    >>> out = ordered_colors()
    >>> len(out) == 5
    True
    >>> colors = {'green', 'orange', 'purple', 'red', 'yellow'}
    >>> set([x[0] for x in out]) == colors
    True
    >>> all([isinstance(x[1], float) for x in out])
    True
    """

    return [('yellow', 0.0), ('orange', .032),('red',.231),('green',.473),('purple',.983)]

In [151]:
out = ordered_colors()
colors = {'green', 'orange', 'purple', 'red', 'yellow'}
all([isinstance(x[1], float) for x in out])

True

In [143]:
colors = skittles.columns.drop('Factory')
for i in colors:
    print(i)
    print(pval_orange(skittles, col = i))

red
0.231
orange
0.032
yellow
0.0
green
0.473
purple
0.983


**Question 6**

Now, suppose you would like to assess whether the two locations make similar amounts of each color overall. That is:
* Combine and count up all the Skittles of each color that were made in Yorkville 
* Combine and count up all the Skittles of each color that were made in Waco

Are these distributions of colors similar? Is the variation among the bags due to each factory making different amounts of each color?

Use a permutation test to assess whether the distribution of colors of Yorkville Skittles is statistically significantly different than those made in Waco. Set a significance level of 0.01 and determine whether you can reject a null hypothesis that answer the question above using a permutation test with 1000 trials. For your test statistic, use the total-variation-distance (TVD).

Create a function `same_color_distribution` of zero variables that outputs a hard-coded tuple with the p-value and whether you 'Fail to Reject' or 'Reject' the null hypothesis.

For this question, the following references may be useful:
* For TVD reference, see [DSC 10](https://www.inferentialthinking.com/chapters/11/2/Multiple_Categories.html) and [Lecture 04](https://github.com/ucsd-ets/dsc80-sp20/tree/master/lectures/04%20-%20Hypothesis%20Testing)
* For permutation test reference, see [DSC 10](https://www.inferentialthinking.com/chapters/12/Comparing_Two_Samples.html) and [Lecture 07](https://github.com/ucsd-ets/dsc80-sp20/tree/master/lectures/07%20-%20Permutation%20Tests)


In [184]:
def same_color_distribution():
    """
    same_color_distribution outputs a hard-coded tuple 
    with the p-value and whether you 'Fail to Reject' or 'Reject' 
    the null hypothesis.

    >>> out = same_color_distribution()
    >>> isinstance(out[0], float)
    True
    >>> out[1] in ['Fail to Reject', 'Reject']
    True
    """
    
    return tuple([0.045,'Reject'])

In [185]:
out = same_color_distribution()
print(isinstance(out[0], float))

out[1] in ['Fail to Reject', 'Reject']

True


True

In [206]:
grouped = skittles.groupby('Factory').sum()
#.diff().loc['Yorkville'].abs().sum() / 2
grouped['sum'] = grouped.sum(axis=1)
grouped = grouped.loc[:,'red':'purple'].div(grouped["sum"], axis=0)
tvd = grouped.diff().loc['Yorkville'].abs().sum() / 2
tvd

0.020404114027914977

In [208]:
def sim_tvd():
    df = skittles.copy()
    df = df.sample(replace = True, frac = 1).reset_index(drop=True)
    grouped = df.groupby('Factory').sum()
    grouped['sum'] = grouped.sum(axis=1)
    grouped = grouped.loc[:,'red':'purple'].div(grouped["sum"], axis=0)
    return grouped.diff().loc['Yorkville'].abs().sum() / 2
    

In [209]:
outs = []
for i in range(10):
    tvds = []
    for i in range(1000):
        tvds.append(sim_tvd())
    outs.append(np.count_nonzero(np.array(tvds) >= tvd) / 1000)
outs

[0.736, 0.72, 0.738, 0.733, 0.749, 0.745, 0.712, 0.732, 0.717, 0.725]

In [200]:
np.mean(outs)

0.0736

### Hypothesis vs Permutation Testing

**Question 7**

In each of the following scenarios, decide  whether  a  permutation test is appropriate to determine if there is a  significant difference between the quantities described. If a permutation test is appropriate, mark 'P'. Otherwise, mark 'H'.

Record your answers in the function `perm_vs_hyp` that outputs a list of length 5, containing the values 'H' and 'P"

1. Compare the DSC 80 pass rate between second years and third years who take the class.
    - Permutation or Hypothesis test?
2. Compare the proportion of Data Science majors who have completed DSC 80 and the proportion of Data Science minors who have completed DSC 80.
    - Permutation or Hypothesis test?
3. Compare the proportion of students who have iPhones to the proportion of students who have Android phones.
    - Permutation or Hypothesis test?
4. Out of the DSC 80 students, the professor asks students whether they prefer DSC 10 or DSC 20.  Compare the proportion of students who prefer DSC 10 to the proportion who prefer DSC 20.
    - Permutation or Hypothesis test?
5. Compare the attendance rate of classes that use iClickers vs classes that do not use iClickers.
    - Permutation or Hypothesis test?

In [210]:
def perm_vs_hyp():
    """
    Multiple choice response for question 8

    >>> out = perm_vs_hyp()
    >>> ans = ['P', 'H']
    >>> len(out) == 5
    True
    >>> set(out) <= set(ans)
    True
    """

    return ['P','P','H','H','P']

In [212]:
out = perm_vs_hyp()
ans = ['P', 'H']
len(out) == 5

set(out) <= set(ans)

True

# Types of Missingness

### Missing by Design (MD)
- The missing field is deliberately missing. The missing field is deliberately not collected or set to null (hence, "missing by design")
- The missingness can be exactly predicted when a column will be null, with only knowledge of the other columns using a function of the rows of the dataset

### Missing Completely at Random (MCAR)
- The missingness of missing value isn't related to the actual, unreported value itself, nor the values in any other fields. The missingness is not systematic.
- The missingness is unconditionally uniform across rows. MCAR doesn't bias the observed data.
- There is no relationship between the missing data and the any of the other data, observed or missing.

### Missing at Random (MAR)
- The missingness of the missing value has nothing to do with the value itself, but may be related to another field.
- The missingness is uniform across rows, perhaps conditional on another column. MAR biases the observed data, but is fixable.
- There is a systematic relationship between the missing values and the observed data (but not the missing values themselves).
- Difference between MD and MAR: If you can *exactly/always* determine missingness on other columns, the missingness is MD. If there is just some sort of systematic relationship between the missing columns/values and other columns/values that may help us predict missingness, the missingness is MAR.

### Non-Ignorable (NI)
- The missingness of the missing value is related to the actual, unreported value.
- NI biases the observed data in unobservable ways.
- There is relationship between the propensity of a value to be missing and its value.

*Note:* In class, we sometimes refer to non-ignorable missingness as "not missing at random (NMAR)"

# Missingness Mechanisms

For each of the following scenarios, choose the most likely mechanism for the missing data in the dataset described.

### After-purchase surveys

**Question 8**

You run a small e-commerce website and send surveys out to customers after they purchase an item from your store. The survey asks whether the customer is satisfied with their purchase ("Yes" or "No"). Below, you are presented with possible datasets, each of which contains a column `satisfied` as described above, as well as a `customer_id` number corresponding to the order. The column `satisfied` is missing data. 

For each of the following datasets, label the column `satisfied` as being `MD`, `MCAR`, `MAR`, `NI`.

1. The dataset consists only of the columns `customer_id` and `satisfied`.
2. The dataset contains the `customer_id` of every customer with an account, even if they didn't make a purchase. Also, in this case, you notice everyone who was sent a survey filled it out.
3. The dataset contains a column specifying if the user later returned the item.
4. The dataset contains a column with the serial number for the item purchased.
5. The dataset contains a column with the price of the item purchased.

Record your answers in the function `after_purchase` that outputs a list of length 5, containing the values `MD`, `MCAR`, `MAR`, `NI`.

In [213]:
def after_purchase():
    """
    Multiple choice response for question 8

    >>> out = after_purchase()
    >>> ans = ['MD', 'MCAR', 'MAR', 'NI']
    >>> len(out) == 5
    True
    >>> set(out) <= set(ans)
    True
    """

    return ['MCAR','MD','MAR','MAR','MAR']

### Miscellaneous missingness questions

**Question 9**

In each of the following scenarios, choose the best answer. Return your answers in a function `multiple_choice`.

1. UCSD has recently adopted GrubHub as the food pre-ordering app for campus restaurants, so you can order your food ahead of time and stop by before your next class. In a table of GrubHub app orders, which contains information such as `restaurant`, `name`, `items`, and `total`, the column `delivery_address` is often missing for UCSD students. The missingness mechanism of these columns is likely:
    - Is the exam grade column `MD`, `MCAR`, `MAR`, `NI`?
1. In a database of student records that records student profile data, such as `name`, `home_address`, `ethnicity`, etc., sometimes the Middle Name column is missing. This column is most likely:
    - `MD`, `MCAR`, `MAR`, `NI`
1. The UCSD Club Basketball team creates a signup sheet for potential new members. The sheet contains the columns: `full_name`, `year`, `email`, `favorite_sports`, `number_of_sports_played`, `sports_previously_played`. The team president notices that many students left the `sports_previously_played` blank. The missingness mechanism of this column is likely:
    - `MD`, `MCAR`, `MAR`, `NI`
1. Associated Students sends out a survey to all students about their 2019 Sun God Experience, with all questions being optional. They notice that many students left the "Were you satisfied with Sun God 2019?". This missingness is most likely:
    - `MD`, `MCAR`, `MAR`, `NI`
1. UC San Diego is implementing two-step login through DUO on October 16th. On October 1st, an administrator creates a table of randomized student codes for all students (not their PID) and the phone numbers associated with their DUO account (two columns total). The administrator notices that there is a lot of missingness in the phone numbers column. This column is most likely:
    - `MD`, `MCAR`, `MAR`, `NI`

In [214]:
def multiple_choice():
    """
    Multiple choice response for question 9

    >>> out = multiple_choice()
    >>> ans = ['MD', 'MCAR', 'MAR', 'NI']
    >>> len(out) == 5
    True
    >>> set(out) <= set(ans)
    True
    >>> out[1] in ans
    True
    """

    return ['MD','MD','MCAR','NI','MAR']

## Congratulations! You're done!

* Submit the lab on Gradescope